<a href="https://colab.research.google.com/github/josemann2007/PortfolioProjects/blob/main/InsurancePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
tr = '/content/drive/MyDrive/Train.csv'

In [2]:
!pip install scikit-learn==1.4.0 feature-engine==1.8.2 mlxtend==0.23.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 26.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
tr = pd.read_csv(tr)

In [5]:
tr.tail()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
12074,ID_ZZA1SES,2010-05-25,2011-05-24,Female,30,2010-05-25,1,NaN,Black,Range Rover,Ibeju/Lekki,Ibeju-Lekki,Car Classic,1
12075,ID_ZZDAC3K,2010-10-03,2011-10-02,Female,59,2010-10-03,1,NaN,NaN,NaN,NaN,NaN,Car Classic,0
12076,ID_ZZIU2XC,2010-10-10,2011-10-08,Male,34,2010-10-10,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
12077,ID_ZZRQ1NF,2010-02-27,2011-02-26,NaN,120,2010-02-27,2,NaN,White,TOYOTA,Victoria Island,Lagos,CVTP,0
12078,ID_ZZWRIIE,2010-07-01,2011-06-30,Female,46,2010-07-01,2,JEEP,Black,Land Rover,Lagos Island,Lagos,Car Classic,0


In [6]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      12079 non-null  object
 1   Policy Start Date       12079 non-null  object
 2   Policy End Date         12079 non-null  object
 3   Gender                  11720 non-null  object
 4   Age                     12079 non-null  int64 
 5   First Transaction Date  12079 non-null  object
 6   No_Pol                  12079 non-null  int64 
 7   Car_Category            8341 non-null   object
 8   Subject_Car_Colour      5117 non-null   object
 9   Subject_Car_Make        9603 non-null   object
 10  LGA_Name                5603 non-null   object
 11  State                   5591 non-null   object
 12  ProductName             12079 non-null  object
 13  target                  12079 non-null  int64 
dtypes: int64(3), object(11)
memory usage: 1.3+ MB


In [7]:
x = tr.drop(['ID','target'], axis=1)
y = tr['target']

In [8]:
x['Policy Start Date'] = pd.to_datetime(x['Policy Start Date'])
x['Policy End Date'] = pd.to_datetime(x['Policy End Date'])
x['First Transaction Date'] = pd.to_datetime(x['First Transaction Date'])

In [9]:
x['No_Pol'] = x['No_Pol'].astype('category')

In [10]:
x['Duration'] = x['Policy End Date'] - x['Policy Start Date']
x['Duration'] = x['Duration'].dt.days

In [11]:
x['Months'] = x['Policy Start Date'].dt.month
#x['Year'] = x['Policy Start Date'].dt.year

In [12]:
x.head()

,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,Duration,Months
0,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,364,5
1,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,364,11
2,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,364,3
3,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,364,8
4,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,124,8


In [13]:
x = x.drop(['Policy Start Date','Policy End Date', 'First Transaction Date'], axis=1)

In [14]:
v = pd.concat([x,y], axis=1)

In [15]:
x.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,Duration,Months
0,Male,30,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,364,5
1,Female,79,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,364,11
2,Male,43,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,364,3
3,Male,2,1,NaN,NaN,NaN,NaN,NaN,CarSafe,364,8
4,Entity,20,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,124,8


In [16]:
from feature_engine.encoding import OneHotEncoder, OrdinalEncoder
from feature_engine.imputation import CategoricalImputer

In [17]:
v = v[v['Age']>=20]

In [18]:
v = v[v['Age']<=80]

In [19]:
v.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11052 entries, 0 to 12078
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Gender              10898 non-null  object  
 1   Age                 11052 non-null  int64   
 2   No_Pol              11052 non-null  category
 3   Car_Category        7789 non-null   object  
 4   Subject_Car_Colour  4696 non-null   object  
 5   Subject_Car_Make    8766 non-null   object  
 6   LGA_Name            4952 non-null   object  
 7   State               4940 non-null   object  
 8   ProductName         11052 non-null  object  
 9   Duration            11052 non-null  int64   
 10  Months              11052 non-null  int32   
 11  target              11052 non-null  int64   
dtypes: category(1), int32(1), int64(3), object(7)
memory usage: 1004.1+ KB


In [20]:
x = v.drop('target', axis=1)
y = v['target']

In [21]:
x['Gender'].value_counts()

,count
Gender,
Male,7157
Female,3149
Joint Gender,212
Entity,146
NOT STATED,143
NO GENDER,57
SEX,34


In [22]:
x['Car_Category'].value_counts()

,count
Car_Category,
Saloon,5676
JEEP,1889
Truck,75
Bus,38
Mini Bus,33
Pick Up,18
Motorcycle,12
Sedan,12
Mini Van,11


In [23]:
len(x['Car_Category'].value_counts())

16

In [24]:
len(x['Subject_Car_Colour'].value_counts())

44

In [25]:
x['Gender'].isnull().sum()

154

In [26]:
x['Car_Category'].isnull().sum()

3263

In [27]:
low_cat = ['Gender']
high_cat = ['Car_Category', 'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State', 'ProductName']

In [28]:
fe1 = CategoricalImputer(imputation_method='frequent', variables=low_cat)
fe2 = CategoricalImputer(imputation_method='missing', fill_value='Missing', variables=high_cat)
fe3 = OneHotEncoder(variables=low_cat, drop_last=True)
fe4 = OrdinalEncoder(encoding_method='ordered', variables=high_cat)

In [29]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([('low_cat_imputer', fe1),
                 ('high_cat_imputer', fe2),
                 ('low_cat_encoder', fe3),
                 ('high_cat_encoder', fe4)])

In [30]:
x.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,Duration,Months
0,Male,30,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,364,5
1,Female,79,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,364,11
2,Male,43,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,364,3
4,Entity,20,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,124,8
5,Male,37,2,NaN,NaN,NaN,NaN,NaN,Car Classic,364,10


In [31]:
x.shape, y.shape

((11052, 11), (11052,))

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [33]:
tr_pro = pipe.fit_transform(x, y)

In [34]:
tr_pro.head()

,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,Duration,Months,Gender_Male,Gender_Female,Gender_Entity,Gender_Joint Gender,Gender_NO GENDER,Gender_NOT STATED
0,30,1,10,28,56,165,79,5,364,5,1,0,0,0,0,0
1,79,1,7,37,56,165,79,5,364,11,0,1,0,0,0,0
2,43,1,10,35,56,165,79,5,364,3,1,0,0,0,0,0
4,20,3,4,24,36,217,93,8,124,8,0,0,1,0,0,0
5,37,2,4,24,36,165,79,5,364,10,1,0,0,0,0,0


In [35]:
from sklearn.feature_selection import SelectKBest, chi2
feat_s = SelectKBest(chi2, k=10)
feat_s = feat_s.fit(tr_pro, y)
tr_pro.columns[feat_s.get_support()]

Index(['Age', 'Car_Category', 'Subject_Car_Colour', 'Subject_Car_Make',
       'LGA_Name', 'State', 'ProductName', 'Duration', 'Months',
       'Gender_Entity'],
      dtype='object')

In [36]:
from sklearn.feature_selection import mutual_info_classif
mutual_info = mutual_info_classif(tr_pro, y)
mutual_info

array([0.01051002, 0.00141562, 0.00469958, 0.0279072 , 0.01861994,
       0.03089372, 0.01925397, 0.05013839, 0.00961126, 0.00217223,
       0.00034661, 0.        , 0.00061979, 0.00115923, 0.        ,
       0.00265717])

In [37]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
rfc = RFE(RandomForestClassifier(), n_features_to_select=10)
rfc = rfc.fit(tr_pro, y)
tr_pro.columns[rfc.get_support()]

Index(['Age', 'No_Pol', 'Car_Category', 'Subject_Car_Colour',
       'Subject_Car_Make', 'LGA_Name', 'State', 'ProductName', 'Duration',
       'Months'],
      dtype='object')

In [38]:
xfs = tr_pro[tr_pro.columns[rfc.get_support()]]

In [39]:
xfs.head()

,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,Duration,Months
0,30,1,10,28,56,165,79,5,364,5
1,79,1,7,37,56,165,79,5,364,11
2,43,1,10,35,56,165,79,5,364,3
4,20,3,4,24,36,217,93,8,124,8
5,37,2,4,24,36,165,79,5,364,10


In [40]:
x_train, x_test, y_train, y_test = train_test_split(xfs, y, test_size=0.25, random_state=53)

In [41]:
sc_tr = StandardScaler()
x_train_sc = sc_tr.fit_transform(x_train)
x_test_sc = sc_tr.transform(x_test)

In [42]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=350, max_depth=6, random_state=53, criterion='gini')
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.8896127397756062


In [43]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C= 1, max_iter =200, verbose=1)
lr.fit(x_train_sc, y_train)
y_pred = lr.predict(x_test_sc)
print(accuracy_score(y_test, y_pred))

0.8863554107853782


In [44]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100, random_state=53)
abc.fit(x_train_sc, y_train)
y_pred = abc.predict(x_test_sc)
print(accuracy_score(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.8878031125588128


In [45]:
!pip install lazypredict

In [46]:
from lazypredict.Supervised import LazyClassifier
lc = LazyClassifier(verbose=1, ignore_warnings=True, custom_metric=None)
models, predictions = lc.fit(x_train, x_test, y_train, y_test)
models[['Accuracy', 'Time Taken']]

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
  3%|▎         | 1/29 [00:00<00:17,  1.61it/s]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.8874411871154542, 'Balanced Accuracy': 0.5575124435698576, 'ROC AUC': 0.5575124435698577, 'F1 Score': 0.8553067787007417, 'Time taken': 0.6215527057647705}


  7%|▋         | 2/29 [00:01<00:13,  2.03it/s]

{'Model': 'BaggingClassifier', 'Accuracy': 0.8552298226565328, 'Balanced Accuracy': 0.5858181888335842, 'ROC AUC': 0.5858181888335842, 'F1 Score': 0.8451624554350865, 'Time taken': 0.39559459686279297}
{'Model': 'BernoulliNB', 'Accuracy': 0.8085414404632646, 'Balanced Accuracy': 0.5950013504649457, 'ROC AUC': 0.5950013504649458, 'F1 Score': 0.819085700496151, 'Time taken': 0.055773258209228516}


 21%|██        | 6/29 [00:01<00:04,  4.92it/s]

...

 97%|█████████▋| 28/29 [00:36<00:01,  1.61s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.8740499457111836, 'Balanced Accuracy': 0.566359918200409, 'ROC AUC': 0.5663599182004091, 'F1 Score': 0.8510642008277152, 'Time taken': 1.3344142436981201}
[LightGBM] [Info] Number of positive: 998, number of negative: 7291
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002798 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 471
[LightGBM] [Info] Number of data points in the train set: 8289, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120401 -> initscore=-1.988643
[LightGBM] [Info] Start training from score -1.988643


100%|██████████| 29/29 [00:37<00:00,  1.29s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.8856315598986608, 'Balanced Accuracy': 0.5715360574140526, 'ROC AUC': 0.5715360574140526, 'F1 Score': 0.8586416063827196, 'Time taken': 0.41213274002075195}


,Accuracy,Time Taken
Model,,
NearestCentroid,0.69,0.09
QuadraticDiscriminantAnalysis,0.67,0.11
GaussianNB,0.67,0.10
PassiveAggressiveClassifier,0.75,0.06
BernoulliNB,0.81,0.06
BaggingClassifier,0.86,0.40
RandomForestClassifier,0.86,2.47
KNeighborsClassifier,0.88,0.88
LGBMClassifier,0.89,0.41


In [47]:
pipe

Pipeline(steps=[('low_cat_imputer',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=['Gender'])),
                ('high_cat_imputer',
                 CategoricalImputer(variables=['Car_Category',
                                               'Subject_Car_Colour',
                                               'Subject_Car_Make', 'LGA_Name',
                                               'State', 'ProductName'])),
                ('low_cat_encoder',
                 OneHotEncoder(drop_last=True, variables=['Gender'])),
                ('high_cat_encoder',
                 OrdinalEncoder(variables=['Car_Category', 'Subject_Car_Colour',
                                           'Subject_Car_Make', 'LGA_Name',
                                           'State', 'ProductName']))])

In [48]:
import pickle
pickle.dump(pipe, open('pipe2.pkl', 'wb'))
pickle.dump(rfc, open('rfc_class.pkl', 'wb'))
pickle.dump(lr, open('lr_class.pkl', 'wb'))
pickle.dump(abc, open('abc_class.pkl', 'wb'))
pickle.dump(sc_tr, open('sc_tr2.pkl', 'wb'))

In [49]:
x['Car_Category'].unique()

array(['Saloon', 'JEEP', nan, 'Truck', 'Bus', 'Motorcycle', 'Mini Bus',
       'Mini Van', 'Van', 'Pick Up', 'Pick Up > 3 Tons', 'CAMRY CAR HIRE',
       'Wagon', 'Shape Of Vehicle Chasis', 'Sedan', 'Station 4 Wheel',
       'Tipper Truck'], dtype=object)

In [50]:
x['No_Pol'].unique()

[1, 3, 2, 4, 5, 10, 6]
Categories (8, int64): [1, 2, 3, 4, 5, 6, 7, 10]

In [51]:
x['Subject_Car_Colour'].unique()

array(['Black', 'Grey', 'Red', nan, 'Blue', 'Silver', 'As Attached',
       'Green', 'Ash', 'White', 'Gray & Gray', 'Wine', 'Brown',
       'Red & Blue', 'Gold', 'Gray & Silver', 'Orange', 'Dark Grey',
       'White & Red', 'Light Green', 'Dark Gray', 'B.Silver', 'Purple',
       'Red & Yellow', 'Yellow', 'Red & White', 'D. Red', 'Dark Red',
       'Black & White', 'White & Blue', 'Cream', 'Dark Green',
       'Red & Black', 'D. Gold', 'Champagne', 'Light Blue',
       'Blue&White&Red', 'White & Yellow', 'Dark Blue', 'Black & Orange',
       'Burgundy', 'Yellow & White', 'Beige Mitalic', 'Light Gray',
       'Blue Sky'], dtype=object)

In [52]:
x['Subject_Car_Make'].unique()

array(['TOYOTA', nan, 'REXTON', 'Lexus', 'Iveco', 'DAF', 'Honda',
       'Mercedes', 'ACURA', 'Ford', 'Volkswagen', 'Nissan', 'Hyundai',
       'Pontiac', '.', 'Range Rover', 'Kia', 'Mitsubishi', 'Scania',
       'BMW', 'Infiniti', 'Renault', 'Volvo', 'Mack', 'Grand Cherokee',
       'Porsche', 'Peugeot', 'Land Rover', 'Mazda', 'Jeep', 'Audi',
       'Suzuki', 'MINI COOPER', 'Chevrolet', 'KA', 'As Attached',
       'Innson', 'Isuzu', 'Skoda', 'Jaguar', 'Chrysler', 'Dodge',
       'Hummer', 'GMC', 'Land Rover.', 'Subaru', 'GAC', 'Fiat', 'Opel',
       'Motorcycle', 'ZOYTE', 'Man', 'Seat', 'Lincoln', 'Rols Royce',
       'CHANGAN', 'LIBERTY', 'ABG', 'Yamaha', 'Black', 'Tata',
       'Ashok Leyland', 'FOTON', 'Geely', 'Raston', 'MG', 'BRILLIANCE',
       'Buik'], dtype=object)

In [53]:
x['LGA_Name'].unique()

array([nan, 'Lagos', 'Ikeja', 'Badagry', 'Eti-Osa', 'Victoria Island',
       'Lagos Mainland', 'Abeokuta', 'Abuja Municipal', 'Yaba', 'Aguda',
       'Surulere', 'Oshodi-Isolo', 'Alimosho', 'Okpe, Delta State',
       'Asaba', 'IbadanCentral', 'Lekki', 'Ibeju/Lekki', 'OBANIKORO',
       'Agege', 'ISHERI', 'Kosofe', 'OGUN', 'Apapa', 'Ibadan South West',
       'Obia/Akpor', 'Ibadan South East', 'Sapele', 'Kaduna South',
       'Ikorodu', 'Amuwo-Odofin', 'Awka', 'GBAGADA', 'Port Harcourt',
       'Minna', 'Shomolu', 'Benin', 'Lagos Island', 'Central',
       'Ebute-Metta', 'Ketu', 'Uyo', 'Mushin', 'Katagum', 'ENUGU EAST',
       'Oredo', 'Festac', 'Ipaja', 'Zaria ', 'Esan West', 'Oluyole',
       'Bwari', 'Obafemi-Owode', 'LGA', 'Orile-Iganmu', 'Egbeda', 'Abuja',
       'Ido', 'Oguta', 'Jos North', 'Ikoyi', 'Ajah', 'Ondo West',
       'Kaduna North', 'Warri', 'Aniocha South', 'Owerri', 'Awka South',
       'Asokoro District', 'Ile-Oluji', 'Port-Harcourt', 'Ifako-Ijaye ',
       'Oshodi'

In [54]:
x['State'].unique()

array([nan, 'Lagos', 'Benue', 'Eti-Osa', 'Ogun', 'Abuja-Municipal',
       'Oshodi-Isolo', 'Delta', 'Ibeju-Lekki', 'Ibadan-West',
       'Obia-Akpor', 'Ibadan-East', 'Kaduna-South', 'Amuwo-Odofin',
       'Anambra', 'Rivers', 'Niger-State', 'Edo', 'Akwa-Ibom',
       'ENUGU-EAST', 'Kaduna', 'Esan-West', 'Obafemi-Owode',
       'Orile-Iganmu', 'Abuja', 'Imo', 'Jos-North', 'Ondo-West',
       'Kaduna-North', 'Aniocha-South', 'Awka-South', 'Ile-Oluji',
       'Port-Harcourt', 'Ifako-Ijaye', 'Nnewi-North', 'Aboh-Mbaise',
       'Oyo', 'Akoko-West', 'Warri-Central', 'Cross-River', 'Jos-South',
       'Onitsha-North', 'Osun', 'Ajeromi-Ifelodun', 'Kano-Municipal',
       'Ife-Central', 'Ilorin-West', 'Lagelu-North', 'Owerri-Municipal',
       'Nsit-Ubium', 'Udi-Agwu', 'Essien-Udim', 'Owerri-West', 'Ondo',
       'Ogbmosho-South', 'Umuahia-South', 'ABULE-EGBA', 'Enugu-North',
       'Ovia-SouthWest', 'Kebbi', 'Ajegunle-State', 'Ekiti-West',
       'Ogun-Waterside', 'Ife-North', 'Ijebu-Ode', 'A

In [55]:
x['ProductName'].unique()

array(['Car Classic', 'Muuve', 'CVTP', 'CarSafe', 'Car Plus',
       'Customized Motor', 'Motor Cycle', 'CarFlex', 'Car Vintage'],
      dtype=object)